In [1]:
// Configuração do ambiente
#include <bits/stdc++.h>

#define endl '\n'

using namespace std;

# Progamação Dinâmica

## Definição

A Programação Dinâmica é um estratégia muito usada em programação competitiva, ela consiste em armazenar resultados de uma função em uma extrutura, visando assim evitar ter que recalcula-los.

Podemos dizer que a programação dinâmica se baseia em:
- Dividir um problema em sub-problemas;
- estabelecer a **relação de recorrência**;
- trocar memória por processamento ao invés de recalcular cada etapa. (**memorização**)

A **memorização** consiste em salvar os resultados já calculados, geralmente em arrays de uma ou mais dimensões. Com isso, <ins>na hora de recalculá-los pode-se acessar a resultado armazenado</ins> , o que é mais rápido do que calcular o resultado novamente.

Outro conceito importante é de **relações de recorrência**, essas relações acontecem quando uma função pode ser escrita <ins>de maneira recursiva</ins>. Podem haver condicionais que afetem como será recursão e sempre deve haver um caso base.

### Quando usar?

Para sabermos se o uso de DP é indicado para a solução de um problema, devemos analisar a existência de **subestruturas ótimas** e **sobreposição**. Esses possuem as seguintes definições:

- **subestruturas ótimas** existem quando pode-se construir uma solução ótima para o problema a partir da solução dos sub-problemas dele.
- **sobreposição** existe se há ao menos um par de entradas distintas que executam o mesmo sub-problema, com o mesmo estado.


### Abordagens

Existem duas abordagens para DP:

- **Top-Down**: seja $x$ um elemento passado para a função $dp()$, essa abordagem irá buscar os valores das dependencias de $x$ na memória, não os encontrando, irá calcula-los.
    - Pode economizar processamento caso a DP seja *esparça*, não há mais calculo do que necessário, é tudo "sob-demanda";
    - Economiza memória se não é necessário calcular todas os estados (DP *esparça*);
    - Dispensa decidir a ordem dos calculos.
    - Pode ser mais lenta se os subproblemas forém revisitados muitas vezes.
- **Botton-Up**: os valores são calculados na ordem que são necessários, dessa forma, ao passar $dp(x)$ já teremos salvo na memória $dp(x-1)$, $dp(x-2)$ e etc. Essa abordagem dispensa a recursão, é calculada a tabela com os dados de maneira iterativa.
    - Pode ter constantes bem menor por dispensar custo da recursão;
    - Permite muitas otimizações (*buscas binárias*, *soma de prefixos*, etc.);
    - Permite economizar memória por saber a ordem de acesso.

Alguns dos algoritmos pensados através de DP são:

- [Knapsack](#Knapsack)
- [Subset Sum](#Subset-Sum)
- [LCS e SCS](#LCS-e-SCS)

## Knapsack

O problema da mochila (Knapsack) é um dos mais clássicos problemas de dp. Generalisando ele podemos entender que <span style="background-color:#d4d4d4">trata-se de encontrar, dada uma coleção de itens com peso e valores, o maior valor somado de um conjunto de itens dentro de um limite de peso.</span>

É possivel encontrar a solução ao analisar-mos a seguinte relação de recorrência:

$$
dp[i,cap]=\begin{cases}
                    0\ \text{se não há mais itens} \\
                    max(dp[i+1,j], dp[i+1, cap - pesos[i]] + valores[i])
                \end{cases}
$$

Nessa relação, `valores` guarda os valores de todos os itens e `pesos` guarda os pesos. O que fazemos é, dado o item item `i` e a capacidade `cap` de peso, analisar qual caminho irá nos retornar o maior valor, se é melhor deixar o item para trás (`dp[i+1,j]`) ou pegar o item (`dp[i+1, cap - pesos[i]] + valores[i]`).

In [2]:
int t[100][100];// Defina os tamanhos confome seu problema, pode usar vector
memset(t, -1, sizeof t); // lembre-se de definir os valores de t com -1;

In [3]:
// ps = pesos, vals = valores, fiz isso por legibilidade
int knapsack(int i, int cap, vector<int> &ps, vector<int> &vals) {
    if(cap < 0) return -0x3f3f3f3f;
    if(i == vals.size()) return 0;
    if(t[i][cap] > -1) return t[i][cap];

    int x = knapsack(i + 1, cap, ps, vals);
    int y = knapsack(i + 1, cap - ps[i], ps, vals) + vals[i];
    return t[i][cap] = max(x, y);
}

Uma adição natural à lógica do problema da mochila é tentar obter o conjuto dos itens "armazenados" na mochila. Encontramos eles da seguinte forma:

In [4]:
unordered_set<int> selecionados; // conjunto dos indices do itens que serão selecionados

In [5]:
// ps = pesos, vals = valores, fiz isso por legibilidade
void retrieve(int i, int cap, vector<int> &ps, vector<int> &vals) {
    if(i == vals.size()) return;

    if(cap >= ps[i]) { // Dividi o if para legibilidade
        if(knapsack(i + 1, cap, ps, vals) < knapsack(i + 1, cap - ps[i], ps, vals) + vals[i]){
            selecionados.insert(i);
            return retrieve(i + 1, cap - ps[i], ps, vals);
        }
    }

    return retrieve(i + 1, cap, ps, vals);
}

In [6]:
vector<int> pesos{5, 4, 2}, valores{500, 300, 250};

cout << knapsack(0, 6, pesos, valores) << endl;

retrieve(0, 6, pesos, valores);
for(auto i : selecionados) cout << i << ' ';
cout << endl;

550
2 1 


## Subset Sum 

**Complexidade:** $O(n 10^{\log c})$

Este é um exemplo de aplicação de DP. <span style="background-color:#d4d4d4">Dado um vetor $p[1..n] \in \mathbb{N}$ e um numero natural $c$, queremos achar um subconjunto $X$ tal que $p(X)=c$.</span>

Obs: $p(X)=\sum_{(i \in X)}p[i]$

Formalisando a lógica, temos:

$$
 t[i,j] =   \begin{cases}
               t[i-1,j] &\text{if } p[i]>j \\
               t[i-1,j] \lor t[i-1,j-p[i]] &\text{if } p[i]\leq j
            \end{cases}
$$

Confira a implementação abaixo, os parametros `p` e `c` são o vetor de valores (peso) e o alvo.

In [7]:
bool subset_sum(const vector<int> &p, const int c) {
    int n = p.size();
    if(c < 0) return false;

    vector t(n + 1, vector<char>(c + 1, false));
    
    for(auto &i : t) i[0] = true;

    for(int i{1}; i <= n; i++) {
        for(int j{1}; j <= c; j++) {
            int b = t[i - 1][j];
            if(j >= p[i - 1]) b = t[i - 1][j] || t[i - 1][j - p[i - 1]];
            t[i][j] = b;
        }
    }

    return t[n][c];
}

In [8]:
vector<int> p{4, 2, 1, 3};
int c{10};

cout << subset_sum(p, c) << endl;

1


In [9]:
bool subset_sum(const vector<int> &p, const int c, set<int> &s) {
    int n = p.size();
    if(c < 0) return false;

    vector t(n + 1, vector<char>(c + 1, false));
    
    for(auto &i : t) i[0] = true;

    for(int i{1}; i <= n; i++) {
        for(int j{1}; j <= c; j++) {
            int b = t[i - 1][j];
            if(j >= p[i - 1]) b = t[i - 1][j] || t[i - 1][j - p[i - 1]];
            t[i][j] = b;
        }
    }

    if (!t[n][c]) return t[n][c];

    int m = n, b = c;
    while (m && b) {
        if(t[m][b] != t[m - 1][b]) {
            s.insert(p[m - 1]);
            b -= p[m - 1];
        }

        m--;
    }

    return t[n][c];
}

In [10]:
vector<int> p2{4, 2, 1, 3};
int c2{7};
set<int> s;
subset_sum(p2, c2, s);

cout << s.size() << endl;
for(auto i : s) cout << i << ' ';
cout << endl;

3
1 2 4 


## LCS e SCS

Para entender o que é LCS (longest common subsequence) e SCS (shortest common supersequence), é necessário entender primeiro que uma <span style="background-color:#d4d4d4">subsequencia é uma sequência presente em 2 ou mais strings na mesma ordem relativa mas **não necessariamente contíguas**</span>. Por outro lado, <span style="background-color:#d4d4d4">Uma supersequência é, dada 2 ou mais strings, a menor string que contenha as outras como subsequência, na mesma ordem relativa</span>.

Por exemplo, dadas as strings `s1 = "ATCG"` e `s2 = "BTGH"`, `lcs(s1, s2) = "TG"` e uma possibilidade de scs é `scs(s1, s2) = "ABTCGH"`.

### LCS

A construção da maior subsequêcia comum é através de uma tabela com valores já calculados para saber o escolha mais vantajosa (Pogramção Dinâmica). A lógica do algoritmo pode ser dividida em 2 etapas, uma que é a construção da tabela dos valores e outra que é a construção da string a partir dessa tábela. Podemos enunciar o algoritmo como:

I. Relação de recorrência na construção da tabela:

$$
    t[i,j] =\begin{cases}
                0 & \text{se } i \geq |s1| \lor j \geq |s2| \\
                t[i+1,j+1] + 1 & \text{se } s1[i] = s2[j] \\
                max(t[i+1,j],t[i,j+1]) & \text{se } s1[i] \neq s2[j]
            \end{cases}
$$

II. Com a tabeça construida, vamos construir da string lcs:

$$
    t[i,j] =\begin{cases}
                ''\ '' & \text{se } i \geq |s1| \lor j \geq |s2| \\
                ''s[i]'' + t[i+1,j+1] & \text{se } s1[i] = s2[j] \\
                t[i+1,j] & \text{se } t[i+1,j] \geq t[i,j+1]\\
                t[i,j+1] & \text{se } t[i+1,j] < t[i,j+1]
            \end{cases}
$$

In [11]:
string lcs(string &s1, string &s2) {
    vector t(s1.size() + 1, vector<int>(s2.size() + 1, 0));

    for(int i = s1.size() - 1; i >= 0; i--) {
        for(int j = s2.size() - 1; j >= 0; j--) {
            if(s1[i] == s2[j]) t[i][j] = t[i + 1][j + 1] + 1;
            else t[i][j] = max(t[i + 1][j], t[i][j + 1]); 
        }
    }

    string s;
    int i = 0, j = 0;

    while(i < s1.size() && j < s2.size()) {
        if(s1[i] == s2[j]) {
            s.push_back(s1[i]);
            i++; j++;
        } else if(t[i + 1][j] >= t[i][j + 1]) i++;
        else j++;
    }

    return s;
}

In [12]:
string s1 = "ATCG", s2 = "BTGH";
lcs(s1, s2)

"TG"

### SCS

A construção da menor super-sequência é, assim como uma lcs, feito a partir de uma tabela com valores que já calculamos. Esses valores nos dizem se é vantajoso adicionar determinado caractere na string agora ou se é melhor adiciona-lo depois. Após construção dessa tabela inicial, contruimos a string. É possivel enunciar as regras de contrução da tabela da seguinte maneira:

I. Relação de recorrência na construção da tabela $t$:


$$
    t[i,j] =\begin{cases}
                j & i = 0\\
                i & j = 0\\
                1 + t[i - 1][j - 1] & s1[i - 1] = s2[j - 1] \\
                1 + min(t[i - 1][j], t[i][j - 1]) & s1[i - 1] \neq s2[j - 1]
            \end{cases}
$$

II. Construção da string `scs`, sendo `i` e `j` iniciados como `i=|s1|`, `j=|s2|` e `k` o tamanho da scs informado por `t[i,j]`, é definida por:

$$
    scs(k, i, j) =\begin{cases}
                \emptyset & k \leq 0\\
                s1(i-1) + scs(k-1,i-1,j-1) & i > 0 \text{ and } j > 0 \text{ and } s1(i-1) = s2(j-1) \\
                s2(j-1) + scs(k-1,i,j-1) & j > 0 \text{ and } t[i,j] - 1 = t[i,j-1] \\
                s1(i-1) + scs(k-1,i-1,j) & i > 0 \text{ and } t[i,j] - 1 \neq t[i,j-1]
            \end{cases}
$$

In [13]:
string scs(string &s1, string &s2) {
    int m = s1.size(), n = s2.size();
    vector t(m + 1, vector<int>(n + 1, 0));

    for(int i{0}; i <= m; i++) {
        for(int j{0}; j <= n; j++) {
            if(!i) t[0][j] = j;
            else if(!j) t[i][0] = i;
            else if(s1[i - 1] == s2[j - 1]) t[i][j] = 1 + t[i - 1][j - 1];
            else t[i][j] = 1 + min(t[i - 1][j], t[i][j - 1]);
        }
    }

    string s = "";
    int len = t[m][n], i = m, j = n;

    while(len--) {
        if(i && j && s1[i - 1] == s2[j - 1]) {
            s += s1[i - 1];
            i--; j--;
        } else if(j && t[i][j] - 1 == t[i][j - 1]) {
            s += s2[j - 1];
            j--;
        } else if(i) {
            s += s1[i - 1];
            i--;
        }
    }

    reverse(s.begin(), s.end());

    return s;
}

In [14]:
scs(s1, s2)

"ABTCGH"

## Fontes

- [Programação Dinâmica 8.1 (video)](https://www.youtube.com/watch?v=KmOIGwEqFWQ)
- [Programação Dinâmica 8.2 (video)](https://youtu.be/IGBXQVZblxg?si=lHFYrpGTNa76_2e8)
- [Subset Sum -- USP](https://www.ime.usp.br/~pf/analise_de_algoritmos/aulas/mochila-subsetsum.html)
- Halim, Steven., Halim, Felix., Effendy, Suhendry. **Competitive Programming 4: The Lower Bound of Programming Contests in the 2020s.** chapter 1-4. Singapore: Lulu Press, Incorporated, 2018.